In [6]:
import glob
import lxml.etree as et
import pandas as pd

element = []
with open("./metadata.tsv") as f:
    header = []
    for line in f:
        line = line.strip()
        if not header:
            header = line.split("\t")
        else:
            element.append(dict(zip(header, line.split("\t"))))

df = pd.DataFrame(element)
df.index = df.Gallica.apply(lambda x: x.split("/")[-1])
for _, row in df.iterrows():
    if not row.Date.isnumeric():
        if "-" in row.Date:
            row.Date = int(sum(map(int, row.Date.split("-"))) / 2)
        else:
            row.Date = "Unknown"
    else:
        row.Date = int(row.Date)

In [7]:
df

,Relation,Catalogue ARK,Repository,Shelfmark,Title,Language,Creator,Date,Digitised by,Source images,Metadata Source,Format,Type,Gallica
Gallica,,,,,,,,,,,,,,
bpt6k12801807,http://catalogue.bnf.fr/ark:/12148/cb386931591,ark:/12148/cb386931591,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","L'Aminte,... pastorale fidellement traduite de...",français,"Tasse, Le",1632,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,12 ff. n. ch. et 158 p. : 6 pl. gr. (en cahier...,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k12801807
bpt6k12801963,http://catalogue.bnf.fr/ark:/12148/cb38650187q,ark:/12148/cb38650187q,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...",Oeuvres de Corneille. Première [-Troisième] pa...,français,"Corneille, Pierre",1652,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,3 vol. : front. daté 1654 et portrait gr. ; In-12,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k12801963
bpt6k12802899,http://catalogue.bnf.fr/ark:/12148/cb302716393,ark:/12148/cb302716393,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","Horace, tragédie (par P. Corneille)",français,"Corneille, Pierre",1647,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,"In-12, 44 ff. non ch., 76 p. (Picot, 18.)",Livre,https://gallica.bnf.fr/ark:/12148/bpt6k12802899
bpt6k12802951,http://catalogue.bnf.fr/ark:/12148/cb386503471,ark:/12148/cb386503471,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...",Cinna ou la Clémence d'Auguste,français,"Corneille, Pierre",1648,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,72 p. ; In-12,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k12802951
bpt6k12802988,http://catalogue.bnf.fr/ark:/12148/cb38650322p,ark:/12148/cb38650322p,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","Horace, tragédie par le Sieur Corneille...",français,"Corneille, Pierre",1647,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,"3 ff. non ch., 65 p. ; In-12",Livre,https://gallica.bnf.fr/ark:/12148/bpt6k12802988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bpt6k9620076z,http://catalogue.bnf.fr/ark:/12148/cb31341744n,ark:/12148/cb31341744n,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","Eudoxe , tragi-comédie par M. de Scudéry",français,"Scudéry, Georges de",1641,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,124 p. : frontisp. gr. ; in-4,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k9620076z
bpt6k9629181d,http://catalogue.bnf.fr/ark:/12148/cb313417528,ark:/12148/cb313417528,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","La mort de César. Edition 2 / , tragédie, par ...",français,"Scudéry, Georges de",1637,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,86 p. : frontisp. gr. ; in-4,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k9629181d
bpt6k9791972w,http://catalogue.bnf.fr/ark:/12148/cb30297741s,ark:/12148/cb30297741s,Bibliothèque nationale de France,"Bibliothèque nationale de France, département ...","La Foire Saint-Germain, comedie . De M. Dancourt",français,Dancourt,1696,Bibliothèque nationale de France,,http://oai.bnf.fr/oai2/OAIHandler?verb=GetReco...,81 p. ; in-12,Livre,https://gallica.bnf.fr/ark:/12148/bpt6k9791972w


In [91]:
import os
from jinja2 import Environment, FileSystemLoader, Template
from dataclasses import dataclass
from xml.sax.saxutils import escape

@dataclass
class Node:
    tag: str
    content: str = ""

    def render(self):
        if self.tag.lower().endswith("sp"):
            tag = "sp"
        elif self.tag == "fw":
            tag = "fw"
        else:
            tag = "p"
        return f"""\n<{tag} type="{self.tag}">{self.content}</{tag}>"""

env = Environment(loader=FileSystemLoader("./"))
template = env.get_template("template.jinja")
ns = dict(namespaces={"a": "http://www.loc.gov/standards/alto/ns-v4#"})

for folder in glob.glob("./todo/books/*"):
    row = df[df.index == os.path.basename(folder)].to_dict(orient='records')[0]

    blocks = []
    for page in sorted(glob.glob(f"{folder}/f*.xml")):
        alto = et.parse(page)
        acceptable_zones = {
            tag.attrib["ID"]: tag.attrib["LABEL"]
            for tag in alto.xpath("//a:OtherTag", **ns)
            if tag.attrib["LABEL"].startswith("MainZone") or tag.attrib["LABEL"] == "text"
        }
        for textblock in alto.xpath("//a:TextBlock", **ns):
            ladas = acceptable_zones.get(textblock.attrib["TAGREFS"], None)
            if ladas is None:
                continue
            content = ""
            for line in textblock.xpath("./a:TextLine", **ns):
                content += "<lb />"
                for child in line.xpath("(a:String | a:SP)", **ns):
                    # print(child)
                    if child.tag.endswith("}SP"):
                        content += " "
                    elif "CONTENT" in child.attrib:
                        content += escape(child.attrib["CONTENT"])
                if content.strip().endswith("¬"):
                    content = content.strip().replace("¬", "")
                else:
                    content += "\n"
            if ladas.endswith("-Continued"):
                if blocks and blocks[-1]:
                    blocks[-1].content += content
                else:
                    blocks.append(Node(tag=ladas, content=content))
            elif ladas == "text":
                if blocks and blocks[-1]:
                    blocks[-1].content += f"<fw>{content}</fw>"
                else:
                    blocks.append(Node(tag="fw", content=content))
            else:
                blocks.append(Node(tag=ladas, content=content))

    content = "".join([content.render() for content in blocks])
    
    rendered_template = template.render(**dict(
        catalog=row["Relation"],
        date=row["Date"],
        author=row["Creator"],
        title=row["Title"],
        content=content
    ))
    with open(f"./todo/tei/{os.path.basename(folder)}.tei.xml", "w") as f:
        f.write(rendered_template)

KeyError: 'TAGREFS'